# Filter Genomic Data with Hail: Chromosomes and Positions

This notebook shows how to filter for chromosomes and positions in genomic data with Hail, and save the filtered Hail Table to an Apollo database (dnax://) on the DNAnexus platform. See documentation for guidance on launch specs for the JupyterLab with Spark Cluster app for different data sizes: https://documentation.dnanexus.com/science/using-hail-to-analyze-genomic-data

Pre-conditions for running this notebook successfully:
- There is an existing Hail MatrixTable in DNAX

## 1) Initiate Spark and Hail

In [ ]:
# Running this cell will output a red-colored message- this is expected.
# The 'Welcome to Hail' message in the output will indicate that Hail is ready to use in the notebook.

from pyspark.sql import SparkSession
import hail as hl

builder = (
    SparkSession
    .builder
    .enableHiveSupport()
)
spark = builder.getOrCreate()
hl.init(sc=spark.sparkContext)

## 2) Read MT

In [5]:
# define MT url

# Note: The MT url should be: `dnax://<database_ID>/<mt_name>`
mt_url = "dnax://database-GFpXJ5j0vzZxPZQ2Ggf14x7q/geno.mt"

In [6]:
# read MT

mt = hl.read_matrix_table(mt_url)

In [ ]:
# View basic properties of MT before filtering
# 
# Note: running 'count()' can be computationally expensive and take
# longer for bigger datasets (these lines can be commented out).

print(f"Num partitions: {mt.n_partitions()}")
print(f"Num loci: {mt.rows().count()}")
mt.describe()

## 3) Filter MT

In this notebook, let's filter for the genomic range of chromosome=1 and position=1512607-6212731.

*Additional documentation on filtering with Hail: https://hail.is/docs/0.2/methods/genetics.html#hail.methods.filter_intervals*

In [10]:
interval = "chr1:1512607-6212731" # format needs to match what is in the MT (i.e. '1:930269-6212731', 'chr1:930269-6212731')

In [11]:
filtered_mt = hl.filter_intervals(mt, [hl.parse_locus_interval(interval, reference_genome="GRCh38"),])

In [ ]:
# View basic properties of MT after filtering
# 
# Note: running 'count()' can be computationally expensive and take
# longer for bigger datasets (these lines can be commented out).

print(f"Num partitions: {filtered_mt.n_partitions()}")
print(f"Num loci: {filtered_mt.rows().count()}")
filtered_mt.describe()

After filtering, we see that the number of loci decreased.

## 4) Create Hail Table from MT and store in DNAX

In [13]:
# Create Hail Table from MT
filtered_tb = filtered_mt.rows()

In [14]:
# Define database and table name
db_name = "database_name"
tb_name = "filtered_chrpos.ht"

In [ ]:
# Create database in DNAX

stmt = f"CREATE DATABASE IF NOT EXISTS {db_name} LOCATION 'dnax://'"
print(stmt)
spark.sql(stmt).show()

In [ ]:
# Store Table in DNAX

import dxpy

# find database ID of newly created database using a dxpy method
db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
url = f"dnax://{db_uri}/{tb_name}"

# Before this step, the Hail Table is just an object in memory. To persist it and be able to access 
# it later, the notebook needs to write it into a persistent filesystem (in this case DNAX).
# See https://hail.is/docs/0.2/hail.Table.html#hail.Table.write for additional documentation.
filtered_tb.write(url) # Note: output should describe size of Table (i.e. number of rows, partitions) 